In [94]:
import pickle
import os
import csv
import pandas as pd
import numpy as np
from collections import defaultdict
from analysis import get_all_tf_metadata, metadata_to_experiments
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

In [96]:
# Architecture: want to have a data loader class 
# and a function which takes in a certain object, 
# ideally relatively standard tensorflow file pointer
# Then constructs this analysis

# Seperate note, for the graph/dataset code, we want to be able 
# to have a data loader and a graph class too. 

In [97]:
my_tags=['auc_train', 'auc_valid', 'auc_test', 'acc_train', 'acc_valid', 'acc_test']
opt_tags=['weight_decay', 'epoch', 'lr', 'train_ratio', 'dataset', 'dropout', 'batch_size', 'pool_graph', 'use_gate', 'num_channel', 'add_self', 'l1_loss_lambda', 'num_layer', 'graph', 'model', 'nb_nodes', 'path']

In [98]:
metadata = get_all_tf_metadata()

In [99]:
df = metadata_to_experiments(metadata)

> /Users/martinweiss/code/conv-graph/analysis/analysis.py(81)metadata_to_experiments()
-> df = df.append(temp_df)
(Pdb) c
> /Users/martinweiss/code/conv-graph/analysis/analysis.py(80)metadata_to_experiments()
-> temp_df = accumulator_to_df(event_accumulator, trial['opts'], trial['hash'], trial['path'])
(Pdb) c
> /Users/martinweiss/code/conv-graph/analysis/analysis.py(81)metadata_to_experiments()
-> df = df.append(temp_df)
(Pdb) c
> /Users/martinweiss/code/conv-graph/analysis/analysis.py(80)metadata_to_experiments()
-> temp_df = accumulator_to_df(event_accumulator, trial['opts'], trial['hash'], trial['path'])
(Pdb) c
> /Users/martinweiss/code/conv-graph/analysis/analysis.py(81)metadata_to_experiments()
-> df = df.append(temp_df)
(Pdb) c
> /Users/martinweiss/code/conv-graph/analysis/analysis.py(80)metadata_to_experiments()
-> temp_df = accumulator_to_df(event_accumulator, trial['opts'], trial['hash'], trial['path'])
(Pdb) c
> /Users/martinweiss/code/conv-graph/analysis/analysis.py(81)met

In [103]:
df

,auc_train,auc_valid,auc_test,acc_train,acc_valid,acc_test,weight_decay,epoch,lr,train_ratio,...,num_channel,add_self,l1_loss_lambda,num_layer,graph,model,nb_nodes,best_epoch,hash,path
0,0.984144,0.930537,0.939468,0.933333,0.855,0.860,0,10,0.001,0.6,...,32,True,0,1,None,mlp,100,9.0,19759111840262d2b340fe140a26cc5b,../experiments/experiments/random/mlp/testing1...
0,0.969858,0.900460,0.870987,0.946667,0.850,0.830,0,10,0.001,0.6,...,32,True,0,1,None,mlp,100,9.0,19759111840262d2b340fe140a26cc5b,../experiments/experiments/random/mlp/testing1...
0,0.943202,0.883591,0.867168,0.943333,0.835,0.850,0,10,0.001,0.6,...,32,True,0,1,None,mlp,100,9.0,19759111840262d2b340fe140a26cc5b,../experiments/experiments/random/mlp/testing1...
0,0.772986,0.730257,0.763944,0.801667,0.690,0.715,0,10,0.001,0.6,...,32,True,0,1,random,slr,100,9.0,071461d122749747a2f201448159e89c,../experiments/experiments/random/slr/testing1...
0,1.000000,0.661064,0.741173,1.000000,0.635,0.690,0,10,0.001,0.6,...,32,True,0,1,None,mlp,1000,9.0,878758a6e47d6f4c3f50b491f8a837ee,../experiments/experiments/random/mlp/testing1...
0,1.000000,0.715899,0.711871,1.000000,0.640,0.650,0,10,0.001,0.6,...,32,True,0,1,None,mlp,1000,9.0,878758a6e47d6f4c3f50b491f8a837ee,../experiments/experiments/random/mlp/testing1...
0,1.000000,0.738295,0.734761,1.000000,0.715,0.665,0,10,0.001,0.6,...,32,True,0,1,None,mlp,1000,9.0,878758a6e47d6f4c3f50b491f8a837ee,../experiments/experiments/random/mlp/testing1...


In [104]:
opt_tags

['weight_decay',
 'epoch',
 'lr',
 'train_ratio',
 'dataset',
 'dropout',
 'batch_size',
 'pool_graph',
 'use_gate',
 'num_channel',
 'add_self',
 'l1_loss_lambda',
 'num_layer',
 'graph',
 'model',
 'nb_nodes',
 'path']

In [105]:
#g1 = df[my_tags].groupby(df['path']).mean()
#g2 = df[opt_tags].groupby(["hash", "path"]).mean()
#group_stats = pd.concat([g1, g2], axis=1, join='inner')
#group_stats
group_stats = df.groupby(['path', 'hash']).mean().reset_index()
group_stats

,path,hash,auc_train,auc_valid,auc_test,acc_train,acc_valid,acc_test,weight_decay,epoch,dropout,batch_size,use_gate,num_channel,add_self,l1_loss_lambda,num_layer,nb_nodes,best_epoch
0,../experiments/experiments/random/mlp/testing1...,19759111840262d2b340fe140a26cc5b,0.965735,0.904863,0.892541,0.941111,0.846667,0.846667,0,10,False,100,0,32,True,0,1,100,9.0
1,../experiments/experiments/random/mlp/testing1...,878758a6e47d6f4c3f50b491f8a837ee,1.000000,0.705086,0.729268,1.000000,0.663333,0.668333,0,10,False,100,0,32,True,0,1,1000,9.0
2,../experiments/experiments/random/slr/testing1...,071461d122749747a2f201448159e89c,0.772986,0.730257,0.763944,0.801667,0.690000,0.715000,0,10,False,100,0,32,True,0,1,100,9.0


In [106]:
for row in group_stats.iterrows():
    path = group_stats.loc[row[0]]['path'] + '/stats.csv'
    np.asarray(row)[1].to_csv(path)